# Step Two (Optional), Constrain the Channels
In this step, you can choose to constrain the channels of your data to focus on specific areas of interest. This is particularly useful when dealing with multi-channel data where only certain channels are relevant for your analysis.

With more channels, data can become noisy and harder to interpret. By constraining the channels, you can reduce this noise and improve the clarity of your results.

We filtered to use the following channels:
``` python
extract_channels = [
    'E1',   # AF3
    'E32',  # AF4
    'E8',   # F3
    'E3',   # Fz
    'E6',   # F4
    'E10',  # FC3
    'E14',  # FCz
    'E21',  # FC4
    'E23',  # C3
    'E31',  # Fz
    'E24',  # C4
    'E47',  # CP3
    'E53',  # CPz
    'E86',  # CP4
    'E55',  # P3
    'E80'   # P4
]
```

This can adjusted to fit your specific needs and the channels that are most relevant to your study.
Usually a good guideline is to select channels in powers of 2 (e.g., 8, 16, 32) to maintain computational efficiency.

In [ ]:
import mne
import os
import glob
import numpy as np

In [ ]:
# load data

# Load all resting state files from raw directory
raw_dir = os.path.join('..', '..', 'processed')

# Find all files with "resting_state" in the name
resting_files = glob.glob(os.path.join(raw_dir, "*resting_state*.fif"))

print(f"Found {len(resting_files)} resting state files:")
for f in resting_files:
    print(f"  - {os.path.basename(f)}")

# load each file and store in a list
raw_list = []
for file_path in resting_files:
    print(f"\nLoading {os.path.basename(file_path)}...")
    raw = mne.io.read_raw_fif(file_path, preload=True)
    raw_list.append(raw)

In [ ]:
# define and extract channels
extract_channels = [
    'E1',   # AF3
    'E32',  # AF4
    'E8',   # F3
    'E3',   # Fz
    'E6',   # F4
    'E10',  # FC3
    'E14',  # FCz
    'E21',  # FC4
    'E23',  # C3
    'E31',  # Fz
    'E24',  # C4
    'E47',  # CP3
    'E53',  # CPz
    'E86',  # CP4
    'E55',  # P3
    'E80'   # P4
]

# sanity check channels
print(f"\nAvailable in first file: {raw_list[0].ch_names}")
available_set = set(raw_list[0].ch_names)
missing = set(extract_channels) - available_set
existing = [ch for ch in extract_channels if ch in available_set]

if missing:
    print(f"Missing: {sorted(missing)}")
    print(f"Available: {len(existing)}/16")

# filter each raw to only existing channels
filtered_raw_list = []
for i, raw in enumerate(raw_list, 1):
    raw_filtered = raw.copy().pick(existing)
    filtered_raw_list.append(raw_filtered)
    if i % 10 == 0:
        print(f"Processed {i}/{len(raw_list)}...")

In [ ]:
# save cleaned data

processed_dir = os.path.join('..', '..', 'filtered_processed')
os.makedirs(processed_dir, exist_ok=True)

# save as FIF files
for i, r in enumerate(filtered_raw_list):
    output_path_fif = os.path.join(processed_dir, f'resting_state_cleaned_{i}.fif')
    r.save(output_path_fif, overwrite=True)
    print(f"Saved as FIF: {output_path_fif}")

# Also save channel names and sampling frequency
for i, r in enumerate(filtered_raw_list):

    metadata = {
        'ch_names': r.ch_names,
        'sfreq': r.info['sfreq'],
        'n_channels': len(r.ch_names),
        'n_times': r.n_times
    }
    metadata_path = os.path.join(processed_dir, f'resting_state_metadata{i}.npy')
    np.save(metadata_path, metadata)
    print(f"Saved metadata: {metadata_path}")